# 1. 라이브러리 불러오기

In [1]:
import os, time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import urllib.request
from selenium.webdriver.common.keys import Keys

# 2. 크롤링 준비

## 상품별 Page URL 반환 함수

In [2]:
# 함수 선언
def PageUrl(itemName):
    item_dic = {"상의":1, "아우터":2, "바지":3, "신발":7} # 입력값으로 index를 반환시킬 dictionary
    url = "https://www.musinsa.com/categories/item/00" + str(item_dic[itemName]) +""
    return url

## 크롬 드라이브 경로 설정 및 불러오기

In [3]:
# 주소 변경
os.chdir("C:/Users/user/Desktop/discover") # 이 파일의 주소로 설정할 것

In [4]:
driver_path = "chromedriver.exe"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--start-maximized') #브라우저 최대화된 상태로 실행
driver = webdriver.Chrome(driver_path, options=chrome_options)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_12060\3221880548.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, options=chrome_options)


# ===================== 수정할 부분 ============================

## 링크로 이동. <span style="color: red;background-color: lightyellow;font-weight: bold">  !!부위 이름 바꾸는 것 주의!!</span>

In [5]:
where = "상의" # 자신이 맡은 파트 기입. 문자열 형태로 넣을 것 [상의, 아우터, 바지, 신발]
driver.get(PageUrl(where))
time.sleep(2)

# ===========================================================

## 컬러 인덱스 제작

#### 옷 부위 별 색상 배열 순서가 달라 각 부위별로 색상 인덱스의 순서를 따로 생성해 저장

In [6]:
html = driver.page_source # html코드 불러오기
bs = BeautifulSoup(html,"html.parser")

# 색상명이 저장돼어있는 class 추출
color = bs.find_all("div", class_ = "tooltip_text")

# 색상 리스트 생성
color_list = []
for color_text in range(len(color)):
    color_list.append(color[color_text].text.strip("\n"))

# 3. 크롤링 시작

In [7]:
# 신상품으로 정렬
driver.find_element("xpath", '//*[@id="goods_list"]/div[2]/div[1]/a[2]/span').click()
time.sleep(1)

# 각 색상별 의류 이미지 저장
for color_index in range(1,len(color_list)+1):
    color_name = color_list[color_index - 1] # 저장할 의류의 색상 저장

    # 계속 다음 색상을 누르면, 옵션이 추가가 되고 기존의 색상이 없어지지 않음. --> 처음이 아닌 경우 두 번 클릭
    if color_index > 1:
        driver.find_element("xpath", '//*[@id="toolTip"]/li[' + str(color_index - 1)+ ']').click()
        time.sleep(1)
        driver.find_element("xpath", '//*[@id="toolTip"]/li[' + str(color_index)+ ']').click()
    else:
        driver.find_element("xpath", '//*[@id="toolTip"]/li[' + str(color_index)+ ']').click()
    time.sleep(1)
    
    
    # 페이지 연속 다운
    ## 스크롤을 내리지 않으면, 의류 이미지가 아닌, 기본 이미지가 저장됨.
    for c in range(0,10):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        
    images = driver.find_elements(By.CSS_SELECTOR, ".li_inner > div > a > img") # 이미지 태그 추출
    
    img_url = [] # 이미지 URL이 저장될 데이터

    # 이미지 URL 저장
    for image in images :
        url = image.get_attribute('src')
        img_url.append(url)
        
    img_folder = './'+where+'/'+ color_name # 폴더 이름 설정
    
    if not os.path.isdir(img_folder) : # 폴더가 없으면 새로 생성하는 조건문 
        os.makedirs(img_folder) #makedirs는 부모 폴더가 없더라도 동시에 생성해줌

    # 이미지 저장
    for index, link in enumerate(img_url) :
        urllib.request.urlretrieve(link, f'' + img_folder+'/'+str(index)+'.jpg')
     
    # 확인용 프린트 문
    count_garbage = 0 # 수집된 기본 이미지 수
    logo_image = 0 # 수집된 의류 이외 이미지 수
    for i in range(len(img_url)):
        if img_url[i] == 'https://image.msscdn.net/images/no_image_125.png':
            count_garbage += 1
        elif "mfile_s01" in img_url[i]:
            logo_image += 1



    print("전체 수집된 이미지 수: " + str(len(img_url)) + " // 수집된 기본 이미지 수: " + str(count_garbage) + " //수집된 의류 이외 이미지 수: " + str(logo_image) + "\n유효 이미지 수: " + str(len(img_url) - count_garbage - logo_image))
    

KeyboardInterrupt: 

# 4. 종료

In [ ]:
driver.quit()